<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/LLM/Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 📘 **LoRA – Low-Rank Adaptation больших языковых моделей**



## 1. Введение

С развитием больших языковых моделей (Large Language Models, LLMs) стало понятно, что их полное обучение требует огромных вычислительных ресурсов. Это делает невозможным перетренировку всей модели для каждой новой задачи или набора данных.

Чтобы решить эту проблему, был предложен ряд **параметрически эффективных методов адаптации (PEFT — Parameter-Efficient Fine-Tuning)**, среди которых **LoRA (Low-Rank Adaptation)** стал особенно популярным благодаря своей простоте и эффективности.



## 2. Что такое LoRA?

**LoRA (Low-Rank Adaptation)** — это метод модификации весов нейронной сети, при котором вместо обновления всех параметров модели добавляются дополнительные **низкоранговые матрицы**, которые и обучаются в процессе настройки.

> 💡 Основная идея LoRA: изменения, необходимые для адаптации модели, имеют **низкий ранг**, поэтому их можно представить как произведение двух маленьких матриц.



## 3. Математическая основа LoRA

### 3.1. Стандартное преобразование

Рассмотрим стандартный слой в нейросети:

$$
y = W \cdot x
$$

Где:
- $ W \in \mathbb{R}^{d \times k} $ — исходная матрица весов,
- $ x \in \mathbb{R}^k $ — входной вектор,
- $ y \in \mathbb{R}^d $ — выходной вектор.

### 3.2. Добавление поправки через LoRA

Вместо того чтобы изменять $ W $ напрямую, мы добавляем к ней поправку $ \Delta W $:

$$
y = (W + \Delta W) \cdot x
$$

### 3.3. Представление поправки как произведение матриц

В классическом fine-tuning $ \Delta W $ может быть такого же размера, как и $ W $, что дорого по памяти.

**В LoRA предлагается представить $ \Delta W $ как произведение двух маленьких матриц:**

$$
\Delta W = A \cdot B
$$

Где:
- $ A \in \mathbb{R}^{d \times r} $
- $ B \in \mathbb{R}^{r \times k} $

Тогда:
- $ \Delta W \in \mathbb{R}^{d \times k} $
- $ r $ — **ранг матрицы**, обычно намного меньше $ d $ и $ k $



## 4. Пример: Вычисление LoRA-поправки

Пусть у нас есть весовая матрица $ W \in \mathbb{R}^{4 \times 3} $:

$$
W =
\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9 \\
10 & 11 & 12
\end{bmatrix}
$$

Выберем ранг $ r = 1 $ и зададим:

$$
A =
\begin{bmatrix}
1 \\
0 \\
-1 \\
2
\end{bmatrix}, \quad
B =
\begin{bmatrix}
2 & -1 & 3
\end{bmatrix}
$$

Вычислим $ \Delta W = A \cdot B $:

$$
\Delta W =
\begin{bmatrix}
1 \\
0 \\
-1 \\
2
\end{bmatrix}
\cdot
\begin{bmatrix}
2 & -1 & 3
\end{bmatrix}
=
\begin{bmatrix}
2 & -1 & 3 \\
0 & 0 & 0 \\
-2 & 1 & -3 \\
4 & -2 & 6
\end{bmatrix}
$$

Итоговая матрица после LoRA:

$$
W_{\text{new}} = W + \Delta W =
\begin{bmatrix}
3 & 1 & 6 \\
4 & 5 & 6 \\
5 & 9 & 6 \\
14 & 9 & 18
\end{bmatrix}
$$



## 5. Обучаемые параметры и экономия

Число обучаемых параметров в LoRA:

$$
\text{params}_{\text{LoRA}} = r \cdot (d + k)
$$

Например, если $ d = 1024 $, $ k = 768 $, $ r = 64 $, то:

$$
\text{params}_{\text{LoRA}} = 64 \cdot (1024 + 768) = 114,688
$$

А всего параметров в $ W $:  
$$
1024 \cdot 768 = 786,432
$$

Таким образом, мы обучаем **всего ~14.6% от общего числа параметров**, сохраняя при этом большую часть функциональности оригинальной модели.



## 6. Почему работает LoRA? Теоретическое обоснование

Существует несколько теоретических обоснований, почему LoRA так эффективен:

### 6.1. Ранг изменений низок

Практические исследования показывают, что изменения, необходимые для адаптации модели под конкретную задачу, имеют **низкий ранг**.

Это связано с тем, что:
- Большие языковые модели уже содержат богатое представление о языке.
- Для специфичной задачи требуется лишь "тонкая настройка" поведения модели.

### 6.2. Малые изменения — стабильность обучения

Малые матрицы $ A $ и $ B $ позволяют избежать дестабилизации модели при дообучении.



## 7. Реализация LoRA в Transformer-архитектурах

В трансформерах LoRA применяется к следующим слоям:

- **Проекции внимания**: `q_proj`, `k_proj`, `v_proj`, `o_proj`
- **Feed-forward сети (FFN)**: `up_proj`, `down_proj`

Пример использования LoRA в коде:

```python
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,                   # ранг
    lora_alpha=16,          # коэффициент масштабирования
    target_modules=["q_proj", "v_proj"],  # куда применить
    lora_dropout=0.1,       # dropout
    bias="none",            # не обучать bias
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
```



## 8. Преимущества LoRA

| Плюс | Описание |
|------|----------|
| 📉 Мало обучаемых параметров | Только ~0.1–1% от общего числа |
| 💾 Экономия памяти | Можно обучать даже на GPU с 8 ГБ видеопамяти |
| 🔄 Быстрая смена задач | Храните разные LoRA-веса для разных задач |
| 📦 Простота интеграции | Совместимость с HuggingFace Transformers, PEFT |
| 🧪 Поддержка обучения с низкой точностью | Например, QLoRA |



## 9. Недостатки LoRA

| Минус | Описание |
|-------|----------|
| ⏳ Может быть менее точным | По сравнению с полным fine-tuning |
| 🎯 Требует настройки | Выбор места внедрения и ранга влияет на качество |
| 🧠 Не всегда применим | На очень специфичных задачах может потребоваться полное обучение |



## 10. Расширения и варианты LoRA

| Название | Описание |
|---------|----------|
| **QLoRA** | LoRA + квантование весов для экономии памяти |
| **DoRA** | Разделение направления и шкалы поправок |
| **AdaLoRA** | Автоматическое определение ранга во время обучения |
| **Multitask LoRA** | Использование одного адаптера для нескольких задач |



## 11. Заключение

LoRA — это мощный и экономичный способ адаптации больших языковых моделей под конкретные задачи. Он позволяет значительно сократить объём вычислений и требования к памяти, сохраняя при этом высокое качество модели. Этот метод стал стандартом де-факто в области **Parameter Efficient Fine-Tuning (PEFT)** и активно используется как в научном сообществе, так и в промышленности.



## 12. Полезные ссылки

- [Оригинальная статья LoRA (Microsoft, 2021)](https://arxiv.org/abs/2106.09685)
- [Hugging Face PEFT Documentation](https://huggingface.co/docs/peft/index)
- [QLoRA Paper](https://arxiv.org/abs/2305.14314)
- [PEFT GitHub Repository](https://github.com/huggingface/peft)
- [LoRA implementations on GitHub](https://github.com/microsoft/LoRA)

